# Rain Prediction Gradient Boosting Classifier


# Import all the libraries

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn.ensemble import GradientBoostingClassifier

import plotly.express as px 
import plotly.graph_objects as go

In [2]:
pd.options.display.max_columns=50

**We ingest the data and derive a few new variables for usage in the model.**

In [3]:
df = pd.read_csv('weatherAUS.csv', encoding='utf-8')

In [6]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,No,No


In [4]:
df = df[pd.isnull(df['RainTomorrow'])==False]

In [5]:
df = df.fillna(df.mean())

In [7]:
df['RainTodayFlag'] = df['RainToday'].apply(lambda x: 1 if x=='Yes' else 0)
df['RainTomorrowFlag'] = df['RainTomorrow'].apply(lambda x: 1 if x=='Yes' else 0)

In [8]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,RainTodayFlag,RainTomorrowFlag
0,2008-12-01,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,No,No,0,0
1,2008-12-02,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,No,No,0,0
2,2008-12-03,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,No,No,0,0
3,2008-12-04,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,No,No,0,0
4,2008-12-05,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,No,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,2017-06-20,Uluru,3.5,21.8,0.0,5.469824,7.624853,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,4.437189,4.503167,9.4,20.9,No,No,0,0
145455,2017-06-21,Uluru,2.8,23.4,0.0,5.469824,7.624853,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,4.437189,4.503167,10.1,22.4,No,No,0,0
145456,2017-06-22,Uluru,3.6,25.3,0.0,5.469824,7.624853,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,4.437189,4.503167,10.9,24.5,No,No,0,0
145457,2017-06-23,Uluru,5.4,26.9,0.0,5.469824,7.624853,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,4.437189,4.503167,12.5,26.1,No,No,0,0


In [9]:
X=df[['WindGustSpeed', 'Humidity3pm']]
y=df['RainTomorrowFlag'].values

# Split data into train and test samples

In [10]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Set model parameters and train (fit) the model

In [11]:
model = GradientBoostingClassifier(loss='deviance',
                               criterion='mse',
                               learning_rate=0.1,
                               subsample=1.0,
                               random_state=0,
                               max_features='sqrt',
                               min_samples_leaf=1000,
                               max_depth=3,
                               n_estimators=500
                              )

In [12]:
clf = model.fit(X_train, y_train)

# Predict class labels on train and test data using our model

In [13]:
pred_labels_tr = model.predict(X_train)

In [14]:
pred_labels_te = model.predict(X_test)

# Generate model summary statistics

In [15]:
print('*************** Tree Summary ***************')
print('No. of classes: ', clf.n_classes_)
print('Classes: ', clf.classes_)
print('No. of features: ', clf.n_features_)
print('No. of Estimators: ', len(clf.estimators_))
print('--------------------------------------------------------')
print("")

*************** Tree Summary ***************
No. of classes:  2
Classes:  [0 1]
No. of features:  2
No. of Estimators:  500
--------------------------------------------------------



In [16]:
print('*************** Evaluation on Test Data ***************')
score_te = model.score(X_test, y_test)
print('Accuracy Score: ', score_te)
print(classification_report(y_test, pred_labels_te))
print('--------------------------------------------------------')
print("")

*************** Evaluation on Test Data ***************
Accuracy Score:  0.834980132916066
              precision    recall  f1-score   support

           0       0.85      0.95      0.90     22067
           1       0.73      0.42      0.53      6372

    accuracy                           0.83     28439
   macro avg       0.79      0.69      0.72     28439
weighted avg       0.82      0.83      0.82     28439

--------------------------------------------------------



In [17]:
print('*************** Evaluation on Training Data ***************')
score_tr = model.score(X_train, y_train)
print('Accuracy Score: ', score_tr)
print(classification_report(y_train, pred_labels_tr))
print('--------------------------------------------------------')

*************** Evaluation on Training Data ***************
Accuracy Score:  0.8328586247516571
              precision    recall  f1-score   support

           0       0.85      0.95      0.90     88249
           1       0.72      0.41      0.53     25505

    accuracy                           0.83    113754
   macro avg       0.79      0.68      0.71    113754
weighted avg       0.82      0.83      0.81    113754

--------------------------------------------------------


# Generate the 3D graph

In [33]:
def Plot_3D(X, X_test, y_test, clf, x1, x2, mesh_size, margin, xe, ye, ze):
            
    # Specify a size of the mesh to be used
    mesh_size=mesh_size
    margin=margin

    # Create a mesh grid on which we will run our model
    x_min, x_max = X.iloc[:, 0].fillna(X.mean()).min() - margin, X.iloc[:, 0].fillna(X.mean()).max() + margin
    y_min, y_max = X.iloc[:, 1].fillna(X.mean()).min() - margin, X.iloc[:, 1].fillna(X.mean()).max() + margin
    xrange = np.arange(x_min, x_max, mesh_size)
    yrange = np.arange(y_min, y_max, mesh_size)
    xx, yy = np.meshgrid(xrange, yrange)
            
    # Calculate predictions on grid
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)

    # Create a 3D scatter plot with predictions
    fig = px.scatter_3d(x=[], y=[], z=[], 
                        width=600, height=600)

    # Set figure title and colors
    fig.update_layout(title_text="Scatter 3D Plot with Gradient Boost Prediction",
                      paper_bgcolor = 'white',
                      scene_camera=dict(up=dict(x=0, y=0, z=1), 
                                        center=dict(x=-0.1, y=0, z=-0.18),
                                        eye=dict(x=xe, y=ye, z=ze)),
                      scene = dict(xaxis=dict(title=x1,
                                              backgroundcolor='white',
                                              color='black',
                                              gridcolor='#f0f0f0',
                                              title_font=dict(size=10),
                                              tickfont=dict(size=10),
                                             ),
                                   yaxis=dict(title=x2,
                                              backgroundcolor='white',
                                              color='black',
                                              gridcolor='#f0f0f0',
                                              title_font=dict(size=10),
                                              tickfont=dict(size=10),
                                              ),
                                   zaxis=dict(title='Probability of Rain Tomorrow',
                                              backgroundcolor='lightgrey',
                                              color='black', 
                                              gridcolor='#f0f0f0',
                                              range=[0.0, 1.0],
                                              title_font=dict(size=10),
                                              tickfont=dict(size=10),
                                              )))

    # Update marker size
    fig.update_traces(marker=dict(size=1))

    # Add prediction plane
    fig.add_traces(go.Surface(x=xrange, y=yrange, z=Z, name='Gradient Boost Prediction',
                              colorscale=['red', 'blue'],
                              reversescale=True,
                              showscale=False, 
                              contours = {"z": {"show": True, "start": 0.5, "end": 0.9, 
                                                "size": 0.5, "color":"white"}}))
    fig.show()
    fig.update_layout(margin=dict(l=0, r=0, t=1, b=1)
                     )
    return fig

In [34]:
fig = Plot_3D(X, X_test, y_test, clf, x1='WindGustSpeed', x2='Humidity3pm', mesh_size=1, margin=10, 
                  xe=-1.7, ye=-1.5, ze=0.4)

# The End!